In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import numpy as np
import gzip
import os

In [3]:
data_dir = './MNIST_data'

In [4]:
def load_mnist():
    with gzip.open(os.path.join(data_dir, 'train-images-idx3-ubyte.gz'), 'rb') as f:
        x_train = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28, 1)
    with gzip.open(os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'), 'rb') as f:
        y_train = np.frombuffer(f.read(), np.uint8, offset=8)
    with gzip.open(os.path.join(data_dir, 't10k-images-idx3-ubyte.gz'), 'rb') as f:
        x_test = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28, 1)
    with gzip.open(os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz'), 'rb') as f:
        y_test = np.frombuffer(f.read(), np.uint8, offset=8)
    return (x_train, y_train), (x_test, y_test)

In [5]:
# mnist.load_data()
# load_mnist()
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [7]:
#Normalize
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [8]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [9]:
#CNN
cnn_model = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten()
])

In [10]:
#LSTM
lstm_model = tf.keras.Sequential([
    layers.LSTM(128),
    layers.Dense(10, activation='softmax')
])

In [11]:
#Hybrid
hybrid_model = tf.keras.Sequential([
    cnn_model,
    layers.Reshape((-1, 1600)),
    lstm_model
])

In [12]:
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


hybrid_model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

1875/1875 [==============================] - 15s 4ms/step - loss: 0.7476 - accuracy: 0.7451 - val_loss: 0.2457 - val_accuracy: 0.9245


In [13]:

#Load from saved
loaded_model = tf.keras.models.load_model('hybrid_model.h5')

OSError: SavedModel file does not exist at: hybrid_model.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
predictions = loaded_model.predict(x_test[:10])
print(np.argmax(predictions, axis=1))